In [6]:
from dotenv import load_dotenv
import os 

load_dotenv(r"../.env")

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
OPENAI_API_KEY[:5]

'sk-pr'

In [7]:
import datasets
from langchain.docstore.document import Document

# Load the dataset
guest_dataset = datasets.load_dataset("agents-course/unit3-invitees", split="train")

# Convert dataset entries into Document objects
docs = [
    Document(
        page_content="\n".join([
            f"Name: {guest['name']}",
            f"Relation: {guest['relation']}",
            f"Description: {guest['description']}",
            f"Email: {guest['email']}"
        ]),
        metadata={"name": guest["name"]}
    )
    for guest in guest_dataset
]

In [8]:
%pip install --upgrade --quiet rank_bm25

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
from langchain_community.retrievers import BM25Retriever
from langchain.tools import Tool 

bm25_retriever = BM25Retriever.from_documents(docs)

def extract_text(query: str) -> str: 
    """Retrieves detailed information about gala guests based on their name or relations."""
    results = bm25_retriever.invoke(query)
    if results:
        return "\n\n".join([doc.page_content for doc in results[:3]])
    else:
        return "No matching guest information found." 
    
guest_info_tool = Tool(
    name="guest_info_retriever", 
    func=extract_text,
    description="Retrieves detailed information about gala guests based on their name or relation."
)

In [10]:
from typing import TypedDict, Annotated
from langgraph.graph.message import add_messages 
from langchain_core.messages import AnyMessage, HumanMessage, AIMessage 
from langgraph.prebuilt import ToolNode
from langgraph.graph import START, StateGraph 
from langgraph.prebuilt import tools_condition 
from langchain_openai import ChatOpenAI 


llm = ChatOpenAI(temperature=0)

chat = llm 
tools = [guest_info_tool]
chat_with_tools = chat.bind_tools(tools)

# Gererate the AgentState and Agent graph

class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages]

def assistant(state: AgentState):
    return {
        "messages": [chat_with_tools.invoke(state["messages"])],
    }

# The graph
builder = StateGraph(AgentState)

# Define nodes: thses do the work
builder.add_node("assistant", assistant)
builder.add_node("tools", ToolNode(tools))

# Define edges: these determine how the control flow moves
builder.add_edge(START, "assistant")
builder.add_conditional_edges("assistant", tools_condition,)
builder.add_edge("tools", "assistant")
alfred = builder.compile()

messages = [HumanMessage(content="Tell me about our guest named 'Lady Ada Lovelace'.")]
response = alfred.invoke({"messages": messages})

print("Alfred's Response:")
print(response['messages'][-1].content)

Alfred's Response:
Lady Ada Lovelace is your best friend. She is an esteemed mathematician and renowned for her pioneering work in mathematics and computing. She is often celebrated as the first computer programmer due to her work on Charles Babbage's Analytical Engine. You can reach her at ada.lovelace@example.com.


In [11]:
%pip install -qU duckduckgo-search langchain-community

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [12]:
# Web search
from langchain_community.tools import DuckDuckGoSearchRun 

search_tool = DuckDuckGoSearchRun()
results = search_tool.invoke("Who's the current President of France?")
print(results)

Emmanuel Macron is a French banker and politician who was elected president of France in 2017. Macron was the first person in the history of the Fifth Republic to win the presidency without the backing of either the Socialists or the Gaullists, and he was France's youngest head of state since Napoleon. PARIS (AP) — French President Emmanuel Macron vowed Thursday to stay in office until the end of his term, due in 2027, and announced that he will name a new prime minister within days following ... Find out who the current president of France is, his political career, his actions, and his impact on the country. Stay informed about French news and presidential decisions. French President Emmanuel Macron has unveiled his new government almost three months after a snap general election delivered a hung parliament. The long-awaited new line up, led by Prime Minister ... French President Emmanuel Macron (center left) and French Prime Minister Michel Barnier (center right) meet with members of

In [13]:
# Creating a Custom tool for weather information
# For the example, we will use a dummy API

from langchain.tools import Tool 
import random 

def get_weather_info(location: str) -> str:
    """Fetches dummy weather information for a given location."""

    weather_conditions = [
        {"condition": "Rainy", "temp_c": 15},
        {"condition": "Sunny", "temp_c": 25},
        {"condition": "Cloudy", "temp_c": 20},
        {"condition": "Snowy", "temp_c": -5},
    ]

    data = random.choice(weather_conditions) 
    return f"Weather in {location}: {data['condition']}, {data['temp_c']}°C"

# Initialize the tool

weather_info_tool = Tool(
    name="weather_info_tool",
    func=get_weather_info, 
    description="Fetches dummy weather information for a given location."
)

In [14]:
# Hub stats tool for infulential AI Builders

from langchain.tools import Tool 
from huggingface_hub import list_models 

def get_hub_stats(author: str) -> str:
    """Fetches the most downloaded model from a specific author on the Hugging Face Hub."""
    try:
        models = list(list_models(author=author, sort="downloads", direction=-1, limit=1))

        if models: 
            model = models[0]
            return f"Most downloaded model by {author} is {model.id} with {model.downloads} downloads." 
        else: 
            return f"No models found for author {author}." 
    except Exception as e: 
        return f"Error fetching models for {author}: {str(e)}" 
    
# Initialize the tool
hub_stats_tool = Tool(
    name="get_hub_stats",
    func=get_hub_stats,
    description="Fetches the most downloaded model from a specific author on the Hugging Face Hub." 
)

print(hub_stats_tool("facebook"))

Most downloaded model by facebook is facebook/esmfold_v1 with 23480812 downloads.


C:\Users\axel.grille\AppData\Local\Temp\ipykernel_15868\2671613754.py:26: LangChainDeprecationWarning: The method `BaseTool.__call__` was deprecated in langchain-core 0.1.47 and will be removed in 1.0. Use :meth:`~invoke` instead.
  print(hub_stats_tool("facebook"))


In [16]:
from typing import TypedDict, Annotated 
from langgraph.graph.message import add_messages 
from langchain_core.messages import AnyMessage, HumanMessage, AIMessage 
from langgraph.prebuilt import ToolNode 
from langgraph.graph import START, StateGraph
from langgraph.prebuilt import tools_condition 
from langchain_openai import ChatOpenAI  

chat = ChatOpenAI(temperature=0)
tools = [search_tool, weather_info_tool, hub_stats_tool] 
chat_with_tools = chat.bind_tools(tools) 

class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages] 
    
    
def assistant(state: AgentState):
    return {
        "messages": [chat_with_tools.invoke(state["messages"])],
    }

# The graph
builder = StateGraph(AgentState)

# Nodes
builder.add_node("assistant", assistant)
builder.add_node("tools", ToolNode(tools)) 

# Edges 
builder.add_edge(START, "assistant")
builder.add_conditional_edges("assistant", tools_condition,) # If the latest message requires a tool, route to tools, else provide a response
builder.add_edge("tools", "assistant")
alfred = builder.compile()


mesages = [HumanMessage(content="Who is Facebook and what's their most popular model?")]

print("Alfred's Response:")
print(response['messages'][-1].content)

Alfred's Response:
Lady Ada Lovelace is your best friend. She is an esteemed mathematician and renowned for her pioneering work in mathematics and computing. She is often celebrated as the first computer programmer due to her work on Charles Babbage's Analytical Engine. You can reach her at ada.lovelace@example.com.
